# Jackknife empirical likelihood analysis of inter-study heterogeneity

An emerging method for meta-analysis that works well with smaller sample sizes is the method of [jackknife empirical likelihood](https://pubmed.ncbi.nlm.nih.gov/34015509/).  Specifically, we use this approach when our goal is to quantify the extent of heterogeneity among a collection of treatment effects that were reported in different research studies.

We begin with the simple meta-analytic model $y_i = \mu + \theta_i + s_i\epsilon_i$, where $y_i$ is an observed (estimated) treatment effect for study $i$, $\mu$ is the common (average) treatement effect, $\theta_i$ is the true, unique treatment effect for study $i$, $s_i$ is the (known) standard error for study $i$, and the $\epsilon_i$ are independent centered and standardized random deviates.  Here we treat the $\theta_i$ as independent random variables with mean zero and variance $\tau^2$.  

If $\tau^2$ is zero, there is no heterogeneity.  Values of $\tau^2$ greater than zero correspond to increasingly heterogeneous treatment effects.  Note that $\tau^2$ is not a standardized measure of heterogeneity, like $\tau^2 / (\tau^2 + {\rm Avg}(s_i^2))$. 

We can unbiasedly estimate $\tau^2$ using $\hat{\tau}^2 = \hat{\rm var}(y_1, \ldots, y_n) - {\rm Avg}(s_i^2)$.  This is a simple estimate to compute, but it is not straightforward to assess it inferentially.  That is, it is difficult to assess how precisely we have estimated $\tau^2$, and whether it is plausible that $\tau^2 = 0$.

To apply jackknife empirical likelihood here, we construct jackknife values $\hat{\tau}_{-i}$ by deleting observation (study) $i$ and recalculating $\hat{\tau}^2$.  We then construct _pseudo-observations_ $\eta_i = n\hat{\tau}^2 - (n-1)\hat{\tau}^2_{-i}$.  Next, we construct a discrete probability distibution on the pseudo-observagtions $\eta_i$ by assigning probability $p_i$ to point $\eta_i$, where $p_i \ge 0$ and $\sum_i p_i = 1$ are imposed.  In addition, we impose the constraint $\sum_i \eta_i p_i = \tau_0^2$, where $\tau_0^2$ is a provisional value of $\tau^2$.  That is, we force the expected value of the empirical likelihood to be exactly equal to $\tau_0^2$.  Subject to these constraints we optimize $\sum_i \log p_i$ which can be seen as a form of nonparametric maximum likelihood.  The value $L(\tau_0^2)$ of $\sum_i \log p_i$ for a given provisional value $\tau_0^2$ is the empirical likelihood at $\tau_0^2$.  We can treat $L$ like a conventional likelihood, so that if $\tau^{2*}$ optimizes $L$, then $2(L(\tau^{2*}) - L(\tau_0^2))$ is a $\chi^2_1$ deviate which tests the hypothesis that the true value of $\tau^2$ is equal to $\tau_0^2$.  The values of $\tau_0^2$ for which the null hypothesis cannot be rejected at level $\alpha$ is a $100 \times (1 - \alpha)\%$ confidence interval for $\tau^2$. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats.distributions as dist
from scipy.optimize import minimize, LinearConstraint

rng = np.random.default_rng()

In the cell below, we write a function that simulated data that could be used for a (mock) meta-analysis.

In [ ]:
def gen_study_dat(n_study, pes, arm_size_mean, arm_size_cv, arm_size_cor, var_cv, clust_icc):
    """
    Simulate data for meta-analysis.  Each study in the meta-analysis is a two arm-study.
        
    Parameters
    ----------
    n_study : number of studies
    pes : population effect size (can be scalar for homogeneous or vector for heterogeneous studies)
    arm_size_mean : the expected sample size of one study arm
    arm_size_cv : the coefficient of variation of study arm sizes
    arm_size_cor : the correlation between effect sizes of the two arms (on copula scale)
    var_cv : the coeffient of variation of the unexplained variance
    clust_icc : if clust_icc, construct 5 study clusters such that there is correlation within the clusters,
                the cluster assignments are returned as 'clust'
  
    Returns
    -------
    md : estimated treatment effects for each study
    sig : estimated (residual) standard deviation for each study
    N1 : sample size for arm 1 in each study
    N2 : sample size for arm 2 in each study
    clust : indicators of study clusters, or None if no clustering is present
  
    Notes
    -----
    The unexplained variance always has mean 1.
    """
    # Generate sample sizes for two arms in each study using a Gaussian copula
    z = rng.normal(size=(n_study, 2))
    z[:, 1] = arm_size_cor*z[:, 0] + np.sqrt(1-arm_size_cor**2)*z[:, 1]
    u = dist.norm.cdf(z)
    v = (arm_size_mean * arm_size_cv)**2
    a = arm_size_mean**2 / v
    b = v / arm_size_mean
    N = dist.gamma(a, scale=b).ppf(u)
    N = np.ceil(N).astype(int)
    N1 = N[:, 0]
    N2 = N[:, 1]
    
    # Now generate standard deviations, centered at 1
    v = var_cv**2
    sig = rng.gamma(1/v, scale=v, size=n_study)
    
    # Convert the standard deviations to standard errors
    f = (N1 + N2) / (N1 * N2)
    se = np.sqrt(sig**2 * f)
    
    z = rng.normal(size=n_study)
    if clust_icc == 0:
        clust = None
    else:
        clust = rng.choice(range(5), n_study)
        for i in range(5):
            jj = np.flatnonzero(clust == i)
            if len(jj) > 0:
                z[jj] = np.sqrt(clust_icc)*rng.normal() + np.sqrt(1 - clust_icc)*z[jj]
    md = pes + z*se
    
    return md, sig, N1, N2, clust

The jackknife empirical likelihood approach described above is implemented in the following two functions.

In [ ]:
def cochran_het_pseudo(md, se):
    """
    Calculate jackknife pseudo-observations for the heterogeneity statistic
    (an unbiased estimate of the variance of study treatment effects) given
    a collection of studies with estimated treatment effects 'md' and standard 
    errors 'se'.
    """
    
    # The heterogeneity statistic
    hetf = lambda md, se: md.var() - (se**2).mean()
    
    h0 = hetf(md, se)
    n = len(md)
    jack = np.zeros(n)
    for j in range(n):
        ii = [i for i in range(n) if i != j]
        h1 = hetf(md[ii], se[ii])
        jack[j] = n*h0 - (n-1)*h1
    return jack

def cochran_het_jel(md, se, npt=25):
    """
    Given a collection of studies with estimated treatment effects 'md' and standard
    errors 'se', construct a grid of 'npt' points for the plausible values of the 
    heterogeneity statistic, then calculate empirical log likelihoods for the points 
    on this grid.
    """
    n = len(md)
    jack = cochran_het_pseudo(md, se)
    bounds = ((0, 1) for _ in range(n))
    A = np.vstack((np.ones(n), jack))

    # Estimate the profile empirical likelihood at these points
    hgrid = np.linspace(jack.min()+1e-4, jack.max()-1e-4, npt)

    # The log-likelihood that we are optimizing.
    f = lambda p: -np.log(np.clip(p, 1e-4, 1)).sum()
    
    # Starting values for the optimization
    start = np.ones(n) / n

    def llf(qs):
        rhs = np.r_[1, qs]
        lc = LinearConstraint(A, rhs, rhs)
        mr = minimize(f, start, method="SLSQP", bounds=bounds, 
                      constraints=lc, options={"maxiter": 5000})
        return mr

    ll = np.zeros(npt)
    for j in range(npt):
        bounds = ((0, 1) for _ in range(n)) # needs to be re-created each time
        mr = llf(hgrid[j])
        if not mr.success:
            # Raise an exception if the optimization fails.
            print("failed: ", hgrid[j])
            print(mr)
            1/0
        ll[j] = -mr.fun

    ll -= ll.max()
    return ll, hgrid

To test the procedure, we first generate test data.

In [ ]:
n = 50 # number of studies

# The true effect sizes in the n studies.  The target of inference is the population
# variance of the values in 'es' (which we call "tau^2").
es = rng.choice([0, 1], size=n)

md, sig, N1, N2, clust = gen_study_dat(n, es, 30, 0.5, 0.7, 0.6, 0)

Next we plot the "profile empirical likellihood function" which reflects whether various potential values of $\tau^2$ can plausibly fit the data.  The higher the profile likelihood, the more plausible the value.  The interval of $\tau^2$ values corresponding to a 95% confidence interval is shown in red.

In [ ]:
n = len(md)
f = (N1 + N2) / (N1 * N2)
se = sig * np.sqrt(f)
ll, qgrid = cochran_het_jel(md, se)

tr = dist.chi2(1).ppf(0.95)

ii = np.flatnonzero(qgrid >= 0)
qgrid = qgrid[ii]
ll = ll[ii]

plt.plot(qgrid, ll)
ii = ll > -tr
plt.plot(qgrid[ii], ll[ii], color="red")
plt.ylabel("Profile empirical likelihood")
plt.xlabel(r"$\tau^2$")
plt.grid(True)